In [1]:
import pandas as pd
from datetime import datetime, timedelta
pd.options.mode.copy_on_write = True 

In [2]:
#Lit le fichier des gares et filtrent sur les gares qui accueillent des voyageurs
liste_city = pd.read_csv("./data_cleaned/stop_location_infos.csv", sep=',')

In [3]:
#Lit le fichier de stops
liste_stops_ter = pd.read_csv("./export-ter-gtfs-last/stops.csv", sep=",")
liste_stops_ter = liste_stops_ter[[str.strip(x[:18]) == "StopPoint:OCETrain" for x in liste_stops_ter["stop_id"]]]
liste_stops_ter = pd.merge(liste_stops_ter, liste_city, how="left", on="stop_id")


In [4]:
# Lit le fichier des temps d'arrêts des trains
liste_stop_times_ter = pd.read_csv("./export-ter-gtfs-last/stop_times.csv", sep=",")

liste_stop_times_ter = liste_stop_times_ter[[str.strip(str(x)[:18]) == "StopPoint:OCETrain" for x in liste_stop_times_ter["stop_id"]]]
liste_stop_times_ter = pd.merge(liste_stop_times_ter, liste_stops_ter[["stop_id", "city"]], how="inner", on="stop_id")
liste_stop_times_ter.to_csv("data_cleaned/stop_times_ter.csv")

In [8]:
liste_stop_times_ter.columns

Index(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled',
       'city'],
      dtype='object')

Dans cette étape, on fusionne la table des trips avec les calendriers pour avoir la liste des dates à laquelles les trains circulent
Ensuite on fusionne ce nouveau Dataframe avec stop_times_ter pour lier les voyages avec leurs gares d'arrêts et leurs horaires

In [9]:
liste_trips_ter = pd.read_csv("./export-ter-gtfs-last/trips.csv", sep=",")
liste_calendar_ter = pd.read_csv("./export-ter-gtfs-last/calendar_dates.csv", sep=",")
print(len(liste_trips_ter))

liste_trips_ter = pd.merge(liste_trips_ter, liste_calendar_ter[["service_id", "date"]], how="inner", on="service_id")

print(len(liste_trips_ter))

liste_stop_times_ter = pd.merge(liste_stop_times_ter, liste_trips_ter[["date", "trip_id"]], how="inner", on="trip_id")

print(len(liste_stop_times_ter))

33280
871798
6167036


In [10]:
liste_stop_times_ter["date"] = [datetime(int(str(row.date)[:4]), int(str(row.date)[4:6]),  int(str(row.date)[-2:]), 0 if int(str(row.arrival_time).split(":")[0]) >= 24 else int(str(row.arrival_time).split(":")[0]), int(str(row.arrival_time).split(":")[1])) for row in liste_stop_times_ter.itertuples()]

In [11]:
liste_stop_times_ter.to_csv("data_cleaned/stop_times_ter_full.csv")

In [12]:
liste_stop_times_ter.date.head(5)

0   2025-01-22 05:16:00
1   2025-01-23 05:16:00
2   2025-01-24 05:16:00
3   2025-01-27 05:16:00
4   2025-01-28 05:16:00
Name: date, dtype: datetime64[ns]

In [15]:
len(liste_stop_times_ter[liste_stop_times_ter["city"] == "Not Found"])

2170

In [13]:
# def check_destination(destination, layer):
#     return len(layer.loc[layer["city"] == destination]) > 0

# def path_back_propagation(layers, destination):
#     paths = []
#     city_to_fetch = destination
#     for layer in reversed(layers):
#         data = layer.loc[layer["city"] == city_to_fetch]
#         if len(data) == 0:
#             return "error"
#         data = data.iloc[0]
#         paths.append(data['current_path'])
#         if data["previous_path"] == "":
#             return paths[::-1]
#         city_to_fetch = data["previous_path"].split(';')[1]


# def filter_direction_trip(data, namesList, start_date=False):
#     find_name = False
#     trip_id = ""
#     end_date = ""
#     default_value = False
#     rows = []

#     if start_date != False:
#         end_date = start_date + timedelta(days=1)
#         default_value = True

#     data = data.sort_values(by=["trip_id", "stop_sequence"])

#     for row in data.itertuples():
#         if not default_value:
#             date_str = row.previous_path.split(';')[-1]
#             date = datetime.strptime(date_str, "%d/%m/%Y, %H:%M:%S")
#             start_date = date + timedelta(minutes=5)
#             end_date = start_date + timedelta(days=1)

#         if row.trip_id != trip_id:
#             trip_id = row.trip_id
#             find_name = False

#         if row.city in namesList :#and (row.date >= start_date and row.date <= end_date):
#             find_name = True

#         if find_name:
#             rows.append(row)
 
#     return pd.DataFrame(rows)

# def shortest_path(depart, destination, start_date):
#     layers = []
#     not_founded = 0 # vérifie qu'on a pas trouvé un chemin vers l'arrivée

#     start_date = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
#     end_date = start_date + timedelta(days=1)
#     date_mask = (liste_stop_times_ter["date"] > start_date) & (liste_stop_times_ter["date"] < end_date)

#     #Raccourci le jeu de donnée nécessaire pour la recherche pour une date spécifique
#     period_stop_time = liste_stop_times_ter.loc[date_mask]

#     trips_id = set(period_stop_time.loc[(period_stop_time["city"] == depart)]["trip_id"])
#     layer = period_stop_time.loc[(period_stop_time["trip_id"].isin(trips_id))]
    
#     layer["previous_path"] = ""
#     layer = filter_direction_trip(layer, [depart], start_date)  
#     layer = layer.sort_values(by="date").drop_duplicates(subset=["stop_id"])

    
#     layer["current_path"] = layer["trip_id"] + ";" + layer["city"] + ";" + layer["date"].dt.strftime("%d/%m/%Y, %H:%M:%S")

#     layers.append(layer)

#     while not_founded < 100:
#         if check_destination(destination, layers[-1]):
#             print(path_back_propagation(layers, destination))
#             return "destination trouver"
        
#         filtered_data = period_stop_time.loc[(period_stop_time["city"].isin(layers[-1]["city"].unique()))]

#         paths = {}
#         for row in filtered_data.itertuples():
#             paths[row.trip_id] = row.trip_id + ";" + str(row.city) + ";" + row.date.strftime("%d/%m/%Y, %H:%M:%S")
        
#         trips_id = set(filtered_data["trip_id"]) - set(layers[-1]["trip_id"])
#         layer = period_stop_time.loc[(period_stop_time["trip_id"].isin(trips_id))]
        
#         layer["previous_path"] = [paths[data.trip_id] for data in layer.itertuples()]
#         layer = filter_direction_trip(layer, layers[-1]["city"].unique())

#         layer = layer.sort_values(by="date").drop_duplicates(subset=["stop_id"])
        
        
#         layer["current_path"] = layer["trip_id"] + ";" + layer["city"] + ";" + layer["date"].dt.strftime("%d/%m/%Y, %H:%M:%S")
#         layer.to_csv("data_cleaned/test.csv")
#         layers.append(layer)
        
#         not_founded += 1
#     return "not found excedeed 100 search"

In [14]:
# depart = "METZERAL"
# destination = "L'HERMITAGE"
# shortest_path(depart, destination, "2024-11-19 05:00:00")
